In [2]:
import pandas as pd
import plotly.io as pio
pio.renderers.default='plotly_mimetype+notebook_connected'
import altair as alt
from countrygroups import EUROPEAN_UNION

In [3]:
df = pd.read_csv("./")
df.head()

ParserError: Error tokenizing data. C error: Expected 2 fields in line 15, saw 12


In [ ]:
df = df.rename(columns={'Country of origin':'origin', 'Country of origin (ISO)':'origin_iso', 'Country of asylum':'asylum', 
                        'Country of asylum (ISO)':'asylum_iso', "Refugees under UNHCR's mandate":'refugees', 
                        'Asylum-seekers':'asylum_seekers', 'IDPs of concern to UNHCR':'idps_of_concern', 
                        'Other people in need of international protection':'need_protect', 'Stateless persons':'stateless',
                        'Host Community':'host', 'Others of concern':'other_of_concern'})

In [ ]:
df = df[df.origin != "Unknown "]
df = df[df.Year>=2000]
# df = df[df.refugees>=100]
df = df.reset_index(drop=True)

# df["path"] = " "
# for i in range(df.shape[0]):
#     df.loc[i, "path"] = str(df.origin_iso[i]+"-"+df.asylum_iso[i])

# df.head()

In [ ]:
df_europe = df[(df.asylum_iso).isin(EUROPEAN_UNION)]
df_europe = df_europe.reset_index(drop=True)

In [ ]:
df_europe_ori_mean = df_europe.groupby(["origin"], as_index=False).mean()
top_ori = df_europe_ori_mean.sort_values(by='refugees', ascending=False).head(10)
df_europe_ori_year = df_europe.groupby(["origin", "Year"], as_index=False).mean()
df_europe_ori_year = df_europe_ori_year[(df_europe_ori_year.origin).isin(top_ori.origin)]

In [ ]:
df_europe_asy_mean = df_europe.groupby(["asylum"], as_index=False).mean()
top_asy = df_europe_asy_mean.sort_values(by='refugees', ascending=False).head(10)
df_europe_asy_year = df_europe.groupby(["asylum", "Year"], as_index=False).mean()
df_europe_asy_year = df_europe_asy_year[(df_europe_asy_year.asylum).isin(top_asy.asylum)]

In [ ]:
selection = alt.selection_single(fields=["origin"], name="random")
color = alt.condition(selection, alt.value("red"), alt.value("lightgrey"))

bar1 = (alt.Chart(df_europe_ori_mean)
        .mark_bar()
        .encode(x=alt.X("origin:N", sort=alt.EncodingSortField(field="refugees", op="mean", order="descending")),
                y=alt.Y("mean(refugees):Q"),
                color=color
        )
).transform_window(
    rank='rank(refugees)',
    sort=[alt.SortField("refugees", order="descending")]
).transform_filter(
    (alt.datum.rank < 10)
).add_selection(selection)
bar1.title = "Countries with Most Refugees to Europe"


color1 = alt.condition(selection, alt.Color("origin:N"), alt.value("lightgrey"))
line1 = (alt.Chart(df_europe_ori_year)
        .mark_line().encode(
                x=alt.X('Year:O'),
                y=alt.Y("refugees:Q"),
                color=color1
        )
# ).transform_aggregate(
#     means='mean(refugees_copy)',
#     groupby=['origin_iso']
# ).transform_window(
#     rank='rank(means)',
#     sort=[alt.SortField("means", order="descending")]
# ).transform_filter(
#      (alt.datum.rank < 20)
).add_selection(selection)
line1.title = "Numbers of Refugees to Europe Each Year"

alt.hconcat(bar1, line1)

alt.HConcatChart(...)

In [ ]:
selection2 = alt.selection_single(fields=["asylum"], name="random")
color2 = alt.condition(selection2, alt.value("red"), alt.value("lightgrey"))

bar2 = (alt.Chart(df_europe_asy_mean)
        .mark_bar()
        .encode(x=alt.X("asylum:N", sort=alt.EncodingSortField(field="refugees", op="mean", order="descending")),
                y=alt.Y("mean(refugees):Q"),
                color=color
        )
).transform_window(
    rank='rank(refugees)',
    sort=[alt.SortField("refugees", order="descending")]
).transform_filter(
    (alt.datum.rank < 10)
).add_selection(selection2)
bar2.title = "European Countries with Most Refugees Accepted"


color3 = alt.condition(selection2, alt.Color("asylum:N"), alt.value("lightgrey"))
line2 = (alt.Chart(df_europe_asy_year)
        .mark_line().encode(
                x=alt.X('Year:O'),
                y=alt.Y("refugees:Q"),
                color=color3
        )
# ).transform_aggregate(
#     means='mean(refugees_copy)',
#     groupby=['origin_iso']
# ).transform_window(
#     rank='rank(means)',
#     sort=[alt.SortField("means", order="descending")]
# ).transform_filter(
#      (alt.datum.rank < 20)
).add_selection(selection2)
line2.title = "Numbers of Refugees to Each Europe Countries Each Year"

alt.hconcat(bar2, line2)

alt.HConcatChart(...)